In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [7]:
# Define paths
train_dir = 'D:/Python/python all/shirt-tshirt classification/dataset/train'
val_dir = 'D:/Python/python all/shirt-tshirt classification/dataset/test'

In [8]:
# Image data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [9]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [11]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 20 images belonging to 2 classes.


In [12]:
# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [13]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [14]:
# Add new top layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [15]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

Epoch 1/25


d:\Python\python all\shirt-tshirt classification\shirt_classifier\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.4792 - loss: 2.0865 - val_accuracy: 0.5000 - val_loss: 1.2302
Epoch 2/25
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 918ms/step - accuracy: 0.5625 - loss: 1.1281

d:\Python\python all\shirt-tshirt classification\shirt_classifier\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5625 - loss: 1.1281 - val_accuracy: 0.5000 - val_loss: 1.8624
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5104 - loss: 1.7988 - val_accuracy: 0.5000 - val_loss: 1.0202
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5625 - loss: 0.9194 - val_accuracy: 0.7000 - val_loss: 0.5486
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.6319 - loss: 0.8126 - val_accuracy: 0.5000 - val_loss: 0.8857
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5000 - loss: 0.9874 - val_accuracy: 0.6000 - val_loss: 0.5817
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5625 - loss: 0.8575 - val_accuracy: 0.6000 - val_loss: 0.7227
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.6250 - loss: 0.8901 - val_accuracy: 0.6000 - val_loss: 0.8072
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6701 - loss: 0.6413 - val_accuracy: 0.7000 - val_loss: 0.5719
Epoch 10/25
2/2 ━━━━━━━━━━━

In [16]:
print(train_generator.class_indices)

{'full_sleeve': 0, 'half_sleeve': 1}


In [17]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-8:]:  # Unfreeze the last 8 layers as an example
    layer.trainable = True

# Recompile the model (necessary after unfreezing layers)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=25
)

Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7292 - loss: 0.9112 - val_accuracy: 0.7500 - val_loss: 0.4879
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 857ms/step - accuracy: 0.7812 - loss: 0.5060 - val_accuracy: 0.5000 - val_loss: 0.6860
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 3s/step - accuracy: 0.6181 - loss: 0.6113 - val_accuracy: 0.5000 - val_loss: 0.7232
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 737ms/step - accuracy: 0.5000 - loss: 0.8209 - val_accuracy: 0.5000 - val_loss: 0.6537
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.6632 - loss: 0.6185 - val_accuracy: 0.6500 - val_loss: 0.6349
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 783ms/step - accuracy: 0.6562 - loss: 0.6139 - val_accuracy: 0.7000 - val_loss: 0.6639
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.6562 - loss: 0.6124 - val_accuracy: 0.7000 - val_loss: 0.6084
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 750ms/step - accuracy: 0.7500 - loss: 0.5389 - val_accuracy: 0.8000 - val_loss: 0.5496
Epoc

In [19]:
# Save the model
model.save('full_hal_sleeve_classifier_vgg16.h5')

In [20]:
# Function to classify new images
def classify_image(image_path,model):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    if prediction < 0.5:
        return "full_sleeve"
    else:
        return "half_sleeve"

In [21]:
# Example usage
print(classify_image(r'D:\Python\python all\shirt-tshirt classification\dataset\test\half_sleeve\98.jpg',model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
half_sleeve


In [22]:
def load_saved_model(model_path):
    model = load_model(model_path)
    return model

In [23]:
model = load_saved_model(r"D:\Python\python all\shirt-tshirt classification\full_hal_sleeve_classifier_vgg16.h5")

In [24]:
# Example usage
print(classify_image(r'D:\Python\python all\shirt-tshirt classification\dataset\test\full_sleeve\352.jpg',model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
full_sleeve
